In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf

In [3]:
tf.set_random_seed(777) # reproducibility

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [5]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [6]:
w1 = tf.Variable(tf.truncated_normal([784, 30]))
b1 = tf.Variable(tf.truncated_normal([1, 30]))
w2 = tf.Variable(tf.truncated_normal([30, 10]))
b2 = tf.Variable(tf.truncated_normal([1, 10]))

In [7]:
def sigma(x):
  # sigmoid function
  return tf.div(tf.constant(1.0),
                tf.add(tf.constant(1.0), tf.exp(-x)))

In [8]:
def sigma_prime(x):
  # derivative of the sigmoid function
  return sigma(x) * (1 - sigma(x))

In [9]:
# Forward prop
l1 = tf.add(tf.matmul(X, w1), b1)
a1 = sigma(l1)
l2 = tf.add(tf.matmul(a1, w2), b2)
y_pred = sigma(l2)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [10]:
# diff
assert y_pred.shape.as_list() == Y.shape.as_list()
diff = (y_pred - Y)

In [11]:
# Back prop (chain rule)
d_l2 = diff * sigma_prime(l2)
d_b2 = d_l2
d_w2 = tf.matmul(tf.transpose(a1), d_l2)

In [12]:
d_a1 = tf.matmul(d_l2, tf.transpose(w2))
d_l1 = d_a1 * sigma_prime(l1)
d_b1 = d_l1
d_w1 = tf.matmul(tf.transpose(X), d_l1)

In [13]:
# Updating network using gradients
learning_rate = 0.5
step = [
        tf.assign(w1, w1 - learning_rate * d_w1),
        tf.assign(b1, b1 - learning_rate *
                  tf.reduce_mean(d_b1, reduction_indices = [0])),
        tf.assign(w2, w2 - learning_rate * d_w2),
        tf.assign(b2, b2 - learning_rate *
                  tf.reduce_mean(d_b2, reduction_indices = [0]))
]

In [14]:
# Running and testing the training process
acct_mat = tf.equal(tf.argmax(y_pred, 1), tf.argmax(Y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

In [15]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(10000):
  batch_xs, batch_ys = mnist.train.next_batch(10)
  sess.run(step, feed_dict = {X: batch_xs,
                              Y: batch_ys})
  if i % 1000 == 0:
    res = sess.run(acct_res, feed_dict = {X: mnist.test.images[:1000],
                                          Y: mnist.test.labels[:1000]})
    print(res)

119.0
843.0
882.0
882.0
899.0
903.0
910.0
921.0
913.0
916.0


In [17]:
# Automatic differentiation in TensorFlow
cost = diff * diff
step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)